In [1]:
from main import *
import sqlalchemy as sa


In [2]:
tweet_obj, user_obj = get_files_and_insert()

mentoo_search_2018-01-01_2019-01-01.jsonl
Table inserted: quoted_tweet_mapping
Table inserted: retweeted_tweet_mapping
Table inserted: replied_to_tweet_mapping
Table inserted: url_tweet_mapping
Table inserted: hashtag_tweet_mapping
Empty table: cashtag_tweet_mapping
Table inserted: mention_tweet_mapping
Table inserted: tweet_data
Table inserted: author_data
Table inserted: quoted_tweet_mapping
Table inserted: retweeted_tweet_mapping
Table inserted: replied_to_tweet_mapping
Table inserted: url_tweet_mapping
Table inserted: hashtag_tweet_mapping
Empty table: cashtag_tweet_mapping
Table inserted: mention_tweet_mapping
Table inserted: tweet_data
Table inserted: author_data
Table inserted: quoted_tweet_mapping
Table inserted: retweeted_tweet_mapping
Table inserted: replied_to_tweet_mapping
Table inserted: url_tweet_mapping
Table inserted: hashtag_tweet_mapping
Empty table: cashtag_tweet_mapping
Table inserted: mention_tweet_mapping
Table inserted: tweet_data
Table inserted: author_data
Tabl

ValueError: A string literal cannot contain NUL (0x00) characters.

In [4]:
from sqlalchemy.dialects.postgresql import insert


def insert_on_duplicate(table, conn, keys, data_iter):
    insert_stmt = insert(table.table).values(list(data_iter))
    on_duplicate_key_stmt = insert_stmt.on_conflict_do_nothing()
    conn.execute(on_duplicate_key_stmt)


In [5]:
def insert_do_nothing_on_conflicts(sqltable, conn, keys, data_iter):
    """
    Execute SQL statement inserting data

    Parameters
    ----------
    sqltable : pandas.io.sql.SQLTable
    conn : sqlalchemy.engine.Engine or sqlalchemy.engine.Connection
    keys : list of str
        Column names
    data_iter : Iterable that iterates the values to be inserted
    """
    from sqlalchemy.dialects.postgresql import insert
    from sqlalchemy import table, column
    columns = []
    for c in keys:
        columns.append(column(c))

    if sqltable.schema:
        table_name = '{}.{}'.format(sqltable.schema, sqltable.name)
    else:
        table_name = sqltable.name

    mytable = table(table_name, *columns)

    insert_stmt = insert(mytable).values(list(data_iter))
    do_nothing_stmt = insert_stmt.on_conflict_do_nothing(
        index_elements=['unique_code'])

    conn.execute(do_nothing_stmt)


# pd.to_sql('mytable', con=sql_engine, method=insert_do_nothing_on_conflicts)


In [6]:
# tweet_obj.tables[cur_table].drop_duplicates(subset=['id'], inplace=True)

In [7]:
cur_table = 'tweet_data'
tweet_obj.tables[cur_table].to_sql(
    name=cur_table, con=engine, if_exists='append', index=False, method=insert_on_duplicate)


In [23]:
for cur_table in tweet_obj.tables.keys():
    if len(tweet_obj.tables[cur_table]) == 0:
        print('Empty table: {}'.format(cur_table))
    else:
        print('Table inserted: {}'.format(cur_table))
        tweet_obj.tables[cur_table].to_sql(name=cur_table, con=engine, if_exists='append', index=False, method=insert_on_duplicate)

Table inserted: quoted_tweet_mapping
Table inserted: retweeted_tweet_mapping
Table inserted: replied_to_tweet_mapping
Table inserted: url_tweet_mapping
Table inserted: hashtag_tweet_mapping
Empty table: cashtag_tweet_mapping
Table inserted: mention_tweet_mapping
Table inserted: tweet_data


In [9]:
for cur_table in user_obj.tables.keys():
    user_obj.tables[cur_table].to_sql(
        name=cur_table, con=engine, if_exists='append', index=False, method=insert_on_duplicate)

In [ ]:
# tweet_obj.tables['replied_to_tweet_mapping']


In [9]:
q = """
SELECT * FROM information_schema.columns
WHERE table_name = 'hashtag_tweet_mapping'
"""

In [10]:
table_schema = pd.read_sql(q, engine)

In [13]:
# set(tweet_obj.tables['replied_to_tweet_mapping'].columns)
set(table_schema['column_name'].unique()) == set(tweet_obj.tables['hashtag_tweet_mapping'].columns)

True

In [12]:
q = """
SELECT count(*) FROM
author_data
""" 

In [10]:
q = """
select count(distinct(id)) from author_data
"""

In [13]:
pd.read_sql(q, engine)

,count
0,2092


In [20]:
q = """
SELECT qt.referenced_tweet_id,td.text,td.id
FROM quoted_tweet_mapping  qt
join tweet_data td
    on qt.referenced_tweet_id = td.id
"""

In [17]:
tweet_obj.tables['tweet_data'].id.count()


6761

In [21]:
quote_data = pd.read_sql(q, engine)

In [26]:
q = """
SELECT *
from mention_tweet_mapping
limit 10
"""

metioned_data = pd.read_sql(q, engine)

In [27]:
metioned_data


,tweet_id,mention_user_id,username,start,end
0,1079874246128660483,2308699184,MentalStigmas,3,17
1,1079874246128660483,5988062,TheEconomist,109,122
2,1079857715684564994,4787013172,nitin33K,3,12
3,1079857715684564994,4573405572,MumbaiPolice,14,27
4,1079857715684564994,3041294294,IPS_Association,106,122
5,1079857715684564994,2686770528,_DigitalIndia,123,137
6,1079848933504045056,4787013172,nitin33K,3,12
7,1079848933504045056,4573405572,MumbaiPolice,14,27
8,1079848933504045056,3041294294,IPS_Association,106,122
9,1079848933504045056,2686770528,_DigitalIndia,123,137
